In [ ]:
from processingQst import *
from tqdm import tqdm
from operator import length_hint
import pandas as pd

In [ ]:
df = pd.read_csv("../data/csv/train.csv",delimiter=";")
fragP1 = []
Qst = df["question"]
nbrRepAll = df["nbr_correct_answers"]
RepListIndxAll = df["correct_answers"]

In [ ]:
for cpt in tqdm(range(len(Qst))):
# for cpt in range(len(Qst)):
    if (cpt> 100):
        break
    elif (cpt>=0):
        nbrRep = nbrRepAll[cpt]
        RepListIndx = RepListIndxAll[cpt]
        if (nbrRep!=1):
            RepListIndx= RepListIndx.replace('|', '') 
        qst = Qst[cpt]
        nwQst = ''
        Spword = False

        # Détécter la négation 
        if(isRequestWrongAns(qst)):
            WrongQst = True 
            # print(WrongQst)
        else: 
            WrongQst = False 
            # print(WrongQst)
        # supprimer les parenthèses 
        qst = del_betParenthese(qst)
        # fragementation des questions selon les ","
        fragQst = splitQst(qst)
        for icpt in range(len(fragQst)):
            # Récupérer les fragements qui contiennent les infos médicales uniquement 
            if not (fragQst[icpt]==''):
                nwQst,Spword = recoverMedFrag(fragQst,icpt,nwQst,Spword)
        nwQst = nwQst.lstrip()
        # collect total pour impression dans un fichier 
        nwQstp = ''
        for j in range(nbrRep):
            Ans = df[f"answers.{RepListIndx[j]}"][cpt] 
            Ans = Ans.lstrip()
            if Spword:
                nwQstp = replaceAtFront(Ans) + nwQst + '.'
            else:
                nwQstp = nwQst + ' '+ Ans.lower() + '.' 
            # Appliquer la négation 
            if WrongQst == True:
                nwQstp = appliquer_negation(nwQstp.lower())
            fragP1.append(nwQstp)
            nwQstp = ''

In [ ]:
file = open('./output/CorpusA.txt','w')
for item in fragP1:
    file.write(item+"\n")     
file.close()